In [65]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from tqdm import tqdm

import sys
import os
module_path = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('lstm.ipynb')),'..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from data.loadData import LoadData
from data.preprocess import Preprocess

In [8]:
stock_name = "^GSPC"
date_predict_start = '2020-01-01'
window_length = 30
slide_range = 40
total_windows = 3
slide = 5
dataloader = LoadData(total_windows, window_length)
train_data, test_data =\
    dataloader.load_and_split_data(stock_name, date_predict_start, window_length, slide_range, total_windows)
x_length = 3
y_length = 5
preprocesser = Preprocess()
x_train, y_train =\
    preprocesser.preprocess_data(train_data, x_length, y_length)
x_test, y_test =\
    preprocesser.preprocess_data(test_data, x_length, y_length, slide=slide)

In [80]:
class Lstm:
    def __init__(self):
        pass

    def LSTM_model(self, y_length, input_shape):
        model = Sequential()
        model.add(LSTM(units = 256, return_sequences = True, input_shape = input_shape))
        model.add(Dropout(0.4))
        model.add(LSTM(units = 128, return_sequences = True))
        model.add(Dropout(0.4))
        model.add(LSTM(units = 64))
        model.add(Dropout(0.4))
        model.add(Dense(units=y_length))
        return model
    
    def build_Lstm_model(self, x_train, y_length):
        model = list(range(0, x_train.shape[0]))
        input_shape = (x_train.shape[2],1)
        for i in range(0, x_train.shape[0]):
            model[i] = self.LSTM_model(y_length, input_shape)
            model[i].compile(optimizer='adam', 
                        loss='mean_squared_error')
            # model[i].summary()
        return model
    
    def train_Lstm_model(self, train_data, model, x_train, y_train):
        history = list(range(0, train_data.shape[0]))
        for i in tqdm(range(0, train_data.shape[0])):
            earlystopper = EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 2)
            history[i] = model[i].fit(x_train[i], y_train[i], epochs=25, batch_size=32, 
                                callbacks = [earlystopper], validation_split = 0.2, shuffle=True)
        return history
    
    def predicting_creat_processed_signal_Lstm(self, x_test, y_test, test_data, model):
        processed_signal = np.ndarray([y_test.shape[0], y_test.shape[1], y_test.shape[2]])
        for i in tqdm(range(0, test_data.shape[0])):
            predicted_prices = model[i].predict(x_test[i])
            processed_signal[i] = predicted_prices
            # processed_signal[i][0] = pd.DataFrame(
            #     {'Close': predicted_prices.flatten()}, index=test_data[i].index[:len(predicted_prices.flatten())])
        return processed_signal

In [81]:
lstm = Lstm()

In [68]:
model = lstm.build_Lstm_model(x_train, y_length)

In [56]:
type(model[0])

keras.engine.sequential.Sequential

In [75]:
x_train.shape

(3, 22, 3, 1)

In [73]:
x_test.shape

(3, 5, 3, 1)

In [76]:
y_test.shape

(3, 5, 5, 1)

In [74]:
test_data.shape

(3, 30)

In [57]:
history = lstm.train_Lstm_model(train_data, model, x_train, y_train)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/25
1/1 [==============================] - 7s 7s/step - loss: 0.2562 - val_loss: 0.5788
Epoch 2/25
1/1 [==============================] - 0s 52ms/step - loss: 0.2516 - val_loss: 0.5821
Epoch 3/25
1/1 [==============================] - 0s 56ms/step - loss: 0.2463 - val_loss: 0.5855
Epoch 4/25
1/1 [==============================] - 0s 55ms/step - loss: 0.2417 - val_loss: 0.5893
Epoch 5/25
1/1 [==============================] - 0s 60ms/step - loss: 0.2348 - val_loss: 0.5933
Epoch 6/25
1/1 [==============================] - 0s 55ms/step - loss: 0.2291 - val_loss: 0.5977
Epoch 6: early stopping


 33%|███▎      | 1/3 [00:07<00:14,  7.01s/it]

Epoch 1/25
1/1 [==============================] - 7s 7s/step - loss: 0.5146 - val_loss: 0.5254
Epoch 2/25
1/1 [==============================] - 0s 63ms/step - loss: 0.5021 - val_loss: 0.5138
Epoch 3/25
1/1 [==============================] - 0s 66ms/step - loss: 0.4928 - val_loss: 0.5009
Epoch 4/25
1/1 [==============================] - 0s 54ms/step - loss: 0.4765 - val_loss: 0.4861
Epoch 5/25
1/1 [==============================] - 0s 66ms/step - loss: 0.4615 - val_loss: 0.4692
Epoch 6/25
1/1 [==============================] - 0s 62ms/step - loss: 0.4383 - val_loss: 0.4496
Epoch 7/25
1/1 [==============================] - 0s 66ms/step - loss: 0.4204 - val_loss: 0.4269
Epoch 8/25
1/1 [==============================] - 0s 57ms/step - loss: 0.3959 - val_loss: 0.4013
Epoch 9/25
1/1 [==============================] - 0s 59ms/step - loss: 0.3707 - val_loss: 0.3730
Epoch 10/25
1/1 [==============================] - 0s 61ms/step - loss: 0.3289 - val_loss: 0.3432
Epoch 11/25
1/1 [==============

 67%|██████▋   | 2/3 [00:14<00:07,  7.57s/it]

Epoch 1/25
1/1 [==============================] - 8s 8s/step - loss: 0.2446 - val_loss: 0.6176
Epoch 2/25
1/1 [==============================] - 0s 53ms/step - loss: 0.2419 - val_loss: 0.6071
Epoch 3/25
1/1 [==============================] - 0s 53ms/step - loss: 0.2397 - val_loss: 0.5958
Epoch 4/25
1/1 [==============================] - 0s 67ms/step - loss: 0.2375 - val_loss: 0.5840
Epoch 5/25
1/1 [==============================] - 0s 53ms/step - loss: 0.2354 - val_loss: 0.5717
Epoch 6/25
1/1 [==============================] - 0s 56ms/step - loss: 0.2315 - val_loss: 0.5580
Epoch 7/25
1/1 [==============================] - 0s 63ms/step - loss: 0.2304 - val_loss: 0.5429
Epoch 8/25
1/1 [==============================] - 0s 56ms/step - loss: 0.2283 - val_loss: 0.5261
Epoch 9/25
1/1 [==============================] - 0s 60ms/step - loss: 0.2259 - val_loss: 0.5073
Epoch 10/25
1/1 [==============================] - 0s 58ms/step - loss: 0.2189 - val_loss: 0.4858
Epoch 11/25
1/1 [==============

100%|██████████| 3/3 [00:24<00:00,  8.30s/it]


In [58]:
model

In [82]:
processed_signal = lstm.predicting_creat_processed_signal_Lstm(x_test, y_test, test_data, model)

  0%|          | 0/3 [00:00<?, ?it/s]

1/1 [==============================] - 0s 23ms/step


 67%|██████▋   | 2/3 [00:00<00:00, 16.95it/s]

1/1 [==============================] - 0s 25ms/step


100%|██████████| 3/3 [00:00<00:00, 16.76it/s]


In [83]:
processed_signal

array([[[ 3.13933659e-03, -4.09055501e-03, -1.18128606e-03,
         -2.01720977e-03,  2.48582195e-03],
        [ 1.69082009e-03, -2.20586499e-03, -6.95799419e-04,
         -1.10538665e-03,  1.34525646e-03],
        [-1.12134661e-03,  1.48473680e-03,  4.74623434e-04,
          7.64797907e-04, -9.38555633e-04],
        [ 1.57327275e-03, -2.07748916e-03, -5.67757466e-04,
         -1.03910931e-03,  1.29473174e-03],
        [ 3.42393224e-03, -4.41310182e-03, -1.46273570e-03,
         -2.17277766e-03,  2.60639051e-03]],

       [[ 4.92943916e-03,  1.95069774e-03,  4.69801482e-04,
          6.91317394e-03,  1.81961618e-03],
        [ 1.38907402e-03,  5.39383385e-04,  1.42663310e-04,
          1.94192352e-03,  5.26729680e-04],
        [-3.12233134e-03, -1.22804858e-03, -3.51494877e-04,
         -4.30220785e-03, -1.07499980e-03],
        [-4.25951928e-03, -1.70378445e-03, -4.93453466e-04,
         -5.83103299e-03, -1.38359948e-03],
        [-1.21495582e-03, -4.53594548e-04, -1.34766451e-04,
  